# Per session and per user analysis

Analysis of users.

# Table of Contents

1.  [Preparation](#preparation)

2.  [Constants](#constants)

3.  [Functions](#functions)

# Preparation
<a id=preparation />

In [30]:
%run "../Functions/2. Game sessions.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (24,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Constants
<a id=constants />

In [31]:
perSessionRelevantColumns = ['sessionId', 'serverTime', 'section']

reachEvents = df152[df152['type']=='reach'].loc[:,perSessionRelevantColumns]
deathEvents = df152[df152['type']=='death'].loc[:,perSessionRelevantColumns]

timedSectionsIndex = [
                        'tutorial1.Checkpoint00',
                        'tutorial1.Checkpoint01',
                        'tutorial1.Checkpoint02',
                        'tutorial1.Checkpoint03',
                        'tutorial1.Checkpoint04',
                        'tutorial1.Checkpoint05',
                        'tutorial1.Checkpoint06',
                        'tutorial1.Checkpoint07',
                        'tutorial1.Checkpoint08',
                        'tutorial1.Checkpoint09',
                        'tutorial1.Checkpoint10',
                        'tutorial1.Checkpoint11',
                        'tutorial1.Checkpoint12',
                        'tutorial1.Checkpoint13',
                        'tutorial1.Checkpoint14',
                     ]
timedSectionsReachedColumns = ['serverTime', 'firstReached', 'firstCompletionDuration']
timedSectionsDeathsColumns = ['deathsCount']
eventSectionsColumns = ['count']

# Functions
<a id=functions />

In [32]:
## Comparison between game and Google form performance

In [33]:
# Returns a given session's checkpoints, the first server time at which they were reached, and completion time
def getCheckpointsTimes( sessionId ):
    perSession = reachEvents[reachEvents['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
    timedSections = perSession.groupby("section").agg({ "serverTime": np.min })
    timedSections["firstReached"] = pd.to_datetime(timedSections["serverTime"])
    timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
    return timedSections

In [34]:
# Returns a given session's checkpoints, and death count
def getDeaths( sessionId ):
    perSession = deathEvents[deathEvents['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
    deathsSections = perSession.groupby("section").size().reset_index(name='deathsCount')
    return deathsSections

In [35]:
# Returns a given user's checkpoints, the first server time at which they were reached, and completion time
def getCheckpointsTimesUser( userId ):
    #print("getCheckpointsTimesUser(" + str(userId) + ")")
    
    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # Call getCheckpointsTimes on all sessions associated with user,
    # then merge by taking oldest checkpoint completion
    timedSections = pd.DataFrame(data=None, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
    
    for sessionId in sessionsList['sessionId']:        
        #print("processing user " + str(userId) + " with session " + str(sessionId))
        times = getCheckpointsTimes( sessionId )
        
        # merge
        # for each checkpoint reached, update if necessary
        for checkpointName in times.index:
            if (np.isnan(timedSections['firstReached'][checkpointName]) 
                or (timeSection['firstReached'][checkpointName] < times['firstReached'][checkpointName])
               ):
                timedSections['serverTime'][checkpointName] = times['serverTime'][checkpointName]
                timedSections['firstReached'][checkpointName] = times['firstReached'][checkpointName]
        #print(times)
        #print(type(times))        
    
    # update of firstCompletionDuration
    #timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
    
    return timedSections

In [36]:
def getDeathsUser( userId ):
    #print("getDeathsUser(" + str(userId) + ")")
    
    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # Call getDeaths on all sessions associated with user,
    # then merge by adding
    deathsSections = pd.DataFrame(0, columns=timedSectionsDeathsColumns,index=timedSectionsIndex)
    
    for sessionId in sessionsList['sessionId']:        
        #print("processing user " + str(userId) + " with session " + str(sessionId))
        deaths = getDeaths( sessionId )
        
        # merge
        # for each checkpoint reached, update if necessary
        for index in deaths.index:
            #print("index=" + str(index))
            checkpointName = deaths['section'][index]
            #print("checkpointName=" + str(checkpointName))
            #print("deaths['deathsCount']["+str(index)+"]=" + str(deaths['deathsCount'][index]))
            
            deathsSections['deathsCount'][checkpointName] = deathsSections['deathsCount'][checkpointName] + deaths['deathsCount'][index]
    
    return deathsSections

In [37]:
# Returns a given session's checkpoints, and event count
def getSectionsEvents( eventType, sessionId ):
    events = df152[df152['type']==eventType].loc[:,perSessionRelevantColumns]
    perSession = events[events['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
    sectionsEvents = perSession.groupby("section").size().reset_index(name='count')
    return sectionsEvents

In [38]:
def getUserSectionsEvents( eventType, userId ):
    
    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # Call getSectionsEvents on all sessions associated with user,
    # then merge by adding
    userSectionsEvents = pd.DataFrame(0, columns=eventSectionsColumns,index=timedSectionsIndex)
    
    for sessionId in sessionsList['sessionId']:
        sessionSectionsEvents = getSectionsEvents( eventType, sessionId )
        
        # merge
        # for each checkpoint reached, update if necessary
        for index in sessionSectionsEvents.index:
            checkpointName = sessionSectionsEvents['section'][index]
            userSectionsEvents['count'][checkpointName] = userSectionsEvents['count'][checkpointName] + sessionSectionsEvents['count'][index]
    
    return userSectionsEvents

In [39]:
def getUserSectionsEventsTotal( eventType, userId ):
    events = getUserSectionsEvents( eventType, userId )
    return events.values.sum()

In [40]:
# Returns a given user's unique reached checkpoints
def getUserCheckpoints( userId ):
    #print("getUserCheckpoints(" + str(userId) + ")")

    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # List all 'reach' events with those sessionIds.
    perUser = reachEvents[reachEvents['sessionId'].isin(sessionsList[sessionsList['userId']==userId])]
    perUser = perUser[perUser['section'].str.startswith('tutorial', na=False)]
    return pd.Series(perUser['section'].unique())


def getDiscrepancyGameGForm( userId ):
    gformNonVal = getNonValidatedCheckpoints(userId)
    gformVal = getValidatedCheckpoints(userId)
    gameVal = getUserCheckpoints(userId)
    
    #sorted, unique values in series1 that are not in series2
    #np.setdiff1d(series1.values, series2.values)
    
    #user has answered questions whose answer they haven't seen in the game
    gameNotEnough = pd.Series(np.setdiff1d(gformVal.values, gameVal.values))
    
    #user has not answered questions whose answer they have seen in the game
    gformNotEnough = []
    maxGameVal = ''
    if gameVal.values.size!=0:
        gameVal.values.max()
    for nonVal in gformNonVal.values:
        if nonVal >= maxGameVal:
            gformNotEnough.append(nonVal)    
    gformNotEnough = pd.Series(gformNotEnough)
    
    return (gameNotEnough, gformNotEnough)

In [41]:
simpleEvents = ['complete','configure','craft','death','equip','gotomooc','gotourl','pickup','reach','restart','selectmenu','start','switch','unequip']

# possible events: complete	configure	craft	death	equip	gotomooc	gotourl	pickup	reach	restart	selectmenu	start	switch	unequip
userDataVectorIndex = np.concatenate( ([
                                            #gform
                                            'score',

                                            #game
                                            'sessionsCount',
                                        ],
                                     simpleEvents))

In [42]:
def getUserDataVector( userId ):
    
    columnName = str(userId)
    
    data = pd.DataFrame(0, columns=[columnName],index=userDataVectorIndex)
    
    data[columnName]['score'] = getScore( userId )
    
    data[columnName]['sessionsCount'] = countSessions( df152, userId, False)
    
    for eventName in simpleEvents:
        if eventName == 'equip':
            data[columnName][eventName] = getUserSectionsEventsTotal('add', userId)
        elif eventName == 'unequip':
            data[columnName][eventName] = getUserSectionsEventsTotal('remove', userId)
        else:
            data[columnName][eventName] = getUserSectionsEventsTotal(eventName, userId)
    
    return data